<a href="https://colab.research.google.com/github/farhadhsn8/MLP/blob/master/implementationDynamicMLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Attribute Information:

1. sepal length in cm
2. sepal width in cm
3. petal length in cm
4. petal width in cm

## class:
1. Iris Setosa
2. Iris Versicolour
3. Iris Virginica

In [165]:
import matplotlib.pyplot as plt
from sklearn import datasets
import random
import numpy as np
import math
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [166]:
dd = np.array([2,4,1,3,0,1])
print(all(dd!=math.inf))

True


In [167]:
import numpy
def load_data():
    x1 = numpy.random.randint(-5,-1,100)
    y1 = numpy.random.randint(0,10,100,)
    # print(y)
    data1 = []
    for i in range(100):
        data1.append(([x1[i],y1[i]]))
    # for i in x:
    #     numpy.ap(x,y)

    x2 = numpy.random.randint(1,5,100)
    y2 = numpy.random.randint(0,10,100,)

    data2 = []
    for i in range(100):
        data2.append(([x2[i],y2[i]]))

    data = []
    for i in range(100):
        data.append(data1[i])
        data.append(data2[i])

    target = []
    for i in range(200):
        if i % 2 == 0: target.append([1,0])
        if i % 2 == 1: target.append([0,1])


    dataset = {'data': data, 'target':target}
    return dataset

In [168]:
class MLP:

#   PARAMS = {
#     # enter learning rate :0.1
#     # enter number of layers :3
#     # enter code of function for layer0 =>[ 1.sigmoid  | 2.tanh  | 3.relu | 4.linear ] :4
#     # enter number of Perceptrons for  layer 1 (start layer number from 0) : 2
#     # enter code of function for layer1 =>[ 1.sigmoid  | 2.tanh  | 3.relu | 4.linear ] :4
#     # enter code of function for layer2 =>[ 1.sigmoid  | 2.tanh  | 3.relu | 4.linear ] :4

#     'LEARNING_RATE' : 0.1 ,
#     'NUMBER_OF_LAYRS' : 3 ,
#     'CODE_OF_ACTIVATION_FUNCTIONS' : [4,4,4] ,
#     'NUMBER_OF_PERCEPTRONS_FOR_HIDDEN_LAYERS' : [2]
#  }

  def __init__(self , feacturesOftrainingdata , lablesOftrainingdata , parameters):
    self.parameters = parameters
    self.feacturesOftrainingdata = feacturesOftrainingdata
    self.lablesOftrainingdata = lablesOftrainingdata
    self.slidingHead = 0
    self.etha = self.parameters['LEARNING_RATE']
    self.numberOfLayers = self.parameters['NUMBER_OF_LAYRS']
    self.layers =  np.empty(self.numberOfLayers,dtype=Layer)
    self.makeLayers()
    
    

  def makeLayers(self):
    for i in range(self.numberOfLayers):
      self.layers[i] = Layer(i , self)

  def train(self):
    return self.calculateMLP_outputForRow_k(self.getCurrentFeatureRow())


  def calculateMLP_outputForRow_k(self, X):
    return self.layers[self.numberOfLayers - 1].claculateLayerOutput(X)

  def getCurrentFeatureRow(self):
    return self.feacturesOftrainingdata[self.slidingHead]

  def getCurrentLableRow(self):
    return self.lablesOftrainingdata[self.slidingHead]


  def updateAllWeightsByBackPropagationAlgorithm(self):
    for layer in self.layers[:0:-1]:
      layer.updateLayerWeights( False)
    for layer in self.layers[:0:-1]:
      layer.updateLayerWeights( True)
    self.resetAllcaches()
    

  
  def trainingModel(self, epoch=1):
    for i in range(epoch):
      self.slidingHead =0
      for j in range(self.feacturesOftrainingdata.shape[0]):
        self.updateAllWeightsByBackPropagationAlgorithm()
        self.slidingHead +=1

  def resetAllcaches(self):
    for i in range(self.numberOfLayers):
      self.layers[i].resetOutput()
      for j in range(self.layers[i].numberOfPerceptrons):
        self.layers[i].perceptrons[j].resetDelta()




  def clearAll(self):
      for i in range(self.numberOfLayers):
        self.layers[i].resetOutput()
        for j in range(self.layers[i].numberOfPerceptrons):
          self.layers[i].perceptrons[j].resetDelta()
          for k in range(self.layers[i].perceptrons[j].numberOfInputs):
            self.layers[i].perceptrons[j].inputBranchs[k].setW()
            print(self.layers[i].perceptrons[j].inputBranchs[k].w)
          

      

  



    



#--------------------------------------------------------------------------


class Layer:

  def __init__(self,layerAddress , MLP):
    self.MLP = MLP
    self.layerAddress = layerAddress
    self.numberOfPerceptrons = self.setNumberOfPerceptrons()
    self.activityFunction = ActivityFunction(self)
    self.perceptrons =  np.empty(self.numberOfPerceptrons,dtype=Perceptron)
    self.perceptrons = self.makePerceptrons()
    self.output = np.full((self.numberOfPerceptrons), math.inf)


  def resetOutput(self):
    self.output = np.full((self.numberOfPerceptrons), math.inf)


  def setNumberOfPerceptrons(self):
    if(self.layerAddress == 0 ):
      return  self.MLP.feacturesOftrainingdata.shape[1]
    if(self.layerAddress == self.MLP.numberOfLayers - 1 ):
      return  self.MLP.lablesOftrainingdata.shape[1]
    return self.MLP.parameters['NUMBER_OF_PERCEPTRONS_FOR_HIDDEN_LAYERS'][self.layerAddress-1]

  def makePerceptrons(self):
    perceptrons =  np.empty(self.numberOfPerceptrons,dtype=Perceptron) 
    for i in range( self.numberOfPerceptrons ):
      perceptrons[i] = Perceptron( i , self)
    return perceptrons

  def getPreviosLayer(self):
    return self.layerAddress != 0 and self.MLP.layers[self.layerAddress - 1 ] or -1

  
  def getNextLayer(self):
    return self.layerAddress != self.MLP.numberOfLayers - 1 \
     and self.MLP.layers[self.layerAddress + 1 ] or -1


  def claculateLayerOutput(self,X):     # receive Vector   # return Vector

    if ((any(self.output==math.inf))==False):
      return self.output
    if(self.layerAddress==0):
      X = X
      return X
    else:
      X = self.getPreviosLayer().claculateLayerOutput(X)
    output =  np.empty(self.numberOfPerceptrons)
    for i in range(self.numberOfPerceptrons):
      if(self.layerAddress == 0 ):
        output[i] = X[i]
      else:
        output[i] = self.perceptrons[i].calculatePerceptronOutput(X)
    self.output = output
    return self.output

  def calculateDerivativeOfActivationFunction(self,net):
    return self.activityFunction.calculateDerivative(net)

  def updateLayerWeights(self, hardUpdate = False):
    for perceptron in self.perceptrons:
      perceptron.updatePerceptronWeights(hardUpdate)
      # if (hardUpdate):
        # perceptron.resetDelta()

  



#--------------------------------------------------------------------------





class ActivityFunction:
  
  def __init__(self,layer):
    self.layer = layer
    self.functionType = self.layer.MLP.parameters['CODE_OF_ACTIVATION_FUNCTIONS'][self.layer.layerAddress]
  
  def runActivationFunction(self,x):
    if (self.functionType == 1) :
      return self.sigmoid(x)
    if (self.functionType == 2) :
      return self.tanh(x)
    if (self.functionType == 3) :
      return self.ReLU(x)
    if (self.functionType == 4) :
      return self.linear(x)

  def sigmoid(self, x):
    return 1 / (1 + math.exp(-x))

  def tanh(self , x):
    t=(math.exp(x)-math.exp(-x))/(math.exp(x)+math.exp(-x))
    return t

  def ReLU(self ,x):
    return max(0.0,x)

  def linear(self , x):
    return x

  def calculateDerivative(self , net):
    if (self.functionType == 1) :
      sig = self.sigmoid(net)
      return (1-sig)*sig
    if (self.functionType == 2) :
      return 1 - self.tanh(net)**2
    if (self.functionType == 3) :
      if(net<0):
        return 0
      return 1
    if (self.functionType == 4) :
      return 1



#--------------------------------------------------------------------------


class Perceptron:

  

  def __init__(self , perceptronNumber , layer ):   # [layerAddress  ,  perceptron] 
    self.perceptronNumber = perceptronNumber
    self.layer = layer
    self.numberOfInputs  =  self.getNumberOfInputs()
    self.inputBranchs =  np.empty(self.numberOfInputs,dtype=Layer)
    self.makeInputs()
    self.delta = math.inf

  def resetDelta(self):
    self.delta = math.inf

  def makeInputs(self):
    for i in range(self.numberOfInputs):
      self.inputBranchs[i] = InputBranch(self , i)

  def getNumberOfInputs(self):
    if(self.layer.layerAddress == 0 ):
      return  1
    return self.layer.getPreviosLayer().numberOfPerceptrons + 0  #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  return self.layer.getPreviosLayer().numberOfPerceptrons + 1 

  def calculatePerceptronOutput(self , X):
        net = self.calculatePerceptronNet(X)
        return self.layer.activityFunction.runActivationFunction(net)

        
  def calculatePerceptronNet(self , X):    # X is input feature vector
        y=0
        # DONT FORGET BAIAS
        X = np.concatenate((X, [0]), axis=None)  #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@   X = np.concatenate((X, [1]), axis=None) 
        for i in range(self.numberOfInputs):
          y += self.inputBranchs[i].calculateBranchOutput(X[i])
        return y


  def getDelta(self):
    if(self.delta != math.inf):
      return self.delta
    desiredOutput=0
    if(self.layer.layerAddress==self.layer.MLP.numberOfLayers - 1):
      desiredOutput = self.layer.MLP.getCurrentLableRow()[self.perceptronNumber]
    X = (self.layer.layerAddress == 0)  and self.layer.MLP.getCurrentFeatureRow() or self.layer.getPreviosLayer().claculateLayerOutput(self.layer.MLP.getCurrentFeatureRow())
    self.delta =  self.calculateDelta(X ,desiredOutput)
    return self.delta
    # print(self.layer.layerAddress, self.perceptronNumber,self.delta)

  def calculateDelta(self,X , desiredOutput):  # X is input vector 
    net = self.calculatePerceptronNet(X)
    if(self.layer.layerAddress == self.layer.MLP.numberOfLayers - 1):     # perceptron in output layer
      return self.layer.calculateDerivativeOfActivationFunction(net) * ( desiredOutput - self.calculatePerceptronOutput(X))
    else:       # perceptron in hidden layer
      sigma = 0
      # layerOutput = self.layer.claculateLayerOutput(self.layer.MLP.getCurrentFeatureRow())
      for perceptron in self.layer.getNextLayer().perceptrons:
        sigma += perceptron.inputBranchs[self.perceptronNumber].w * perceptron.getDelta() 
      return self.layer.calculateDerivativeOfActivationFunction(net) * sigma

  
  def updatePerceptronWeights(self,hardUpdate = False):
    for inputBranch in self.inputBranchs:
      hardUpdate and inputBranch.updateW() or inputBranch.updateWnew()


    


  
#--------------------------------------------------------------------------

class InputBranch:
  
  def __init__(self , perceptron, inputNumber):
    self.inputNumber = inputNumber
    self.perceptron = perceptron
    self.setW()
    self.Wnew = self.w

  def setW(self):
    if(self.perceptron.layer.layerAddress == 0):
      self.w =  1
    self.w = 1# random.uniform(-50, 50) #00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

  def calculateBranchOutput(self , x):
    return self.w * x 

  

  def updateWnew(self):
    self.Wnew =self.w +  self.perceptron.layer.MLP.etha * self.perceptron.getDelta() *\
     np.concatenate((self.perceptron.layer.getPreviosLayer().claculateLayerOutput(self.perceptron.layer.MLP.getCurrentFeatureRow()), [0]), axis=None)[self.inputNumber] #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

  def updateW(self):
    self.w = self.Wnew





In [169]:
iris = datasets.load_iris()
features = iris.data  
target = pd.get_dummies(iris.target).to_numpy()
features.shape   # (150, 4)
target.shape

#---------------shuffle---------------------
dataset = np.hstack(( features,target,np.reshape(iris.target,(-1,1))))
from sklearn.utils import shuffle
dataset=shuffle(dataset)

#-------------test & train ---------------
train=dataset[0:120,:]    
test=dataset[120:,:]  
test.shape                #(30, 7)
train.shape              # (120, 7)


# dataset[149]


(120, 8)

In [170]:
ddd = load_data()
s1= ddd['data']
s2= ddd['target']
s1 = np.array(s1)
s2 = np.array(s2)
s2.shape

(200, 2)

In [171]:
PARAMS = {
    # enter learning rate :0.1
    # enter number of layers :3
    # enter code of function for layer0 =>[ 1.sigmoid  | 2.tanh  | 3.relu | 4.linear ] :4
    # enter number of Perceptrons for  layer 1 (start layer number from 0) : 2
    # enter code of function for layer1 =>[ 1.sigmoid  | 2.tanh  | 3.relu | 4.linear ] :4
    # enter code of function for layer2 =>[ 1.sigmoid  | 2.tanh  | 3.relu | 4.linear ] :4
    'LEARNING_RATE' : 0.1 ,
    'NUMBER_OF_LAYRS' : 3 ,
    'CODE_OF_ACTIVATION_FUNCTIONS' : [4,4,4] ,
    'NUMBER_OF_PERCEPTRONS_FOR_HIDDEN_LAYERS' : [2]
 }


PARAMS['NUMBER_OF_PERCEPTRONS_FOR_HIDDEN_LAYERS']



[2]

In [172]:
# IRIS_MLP = MLP(train[:,0:4] ,train[:,4:7] )
# IRIS_MLP = MLP(train[:,0:4] ,5 * np.reshape(train[:,7],(-1,1,1)) )
IRIS_MLP = MLP(np.array([[1,1]]) ,np.array([[2,2]]) , PARAMS )
# IRIS_MLP = MLP(s1 ,s2 )

In [117]:
IRIS_MLP.clearAll()

1
1
1
1
1
1
1
1
1
1


In [176]:
IRIS_MLP.trainingModel(1)

In [177]:
s=0
# for i in test:
  # print([IRIS_MLP.calculateMLP_outputForRow_k(s1[i])[0] ,IRIS_MLP.calculateMLP_outputForRow_k(s1[i])[1]] , list(s2[i]))
  # s+=int([int(IRIS_MLP.calculateMLP_outputForRow_k(s1[i])[0]>0) ,int(IRIS_MLP.calculateMLP_outputForRow_k(s1[i])[1]>0)] == list(s2[i]))
# 

  # print(IRIS_MLP.calculateMLP_outputForRow_k(i[0:4]) , i[4:7])


print(IRIS_MLP.calculateMLP_outputForRow_k([1,1]) )

[1.78062336 1.78062336]


In [140]:
for i in range(IRIS_MLP.numberOfLayers):
  print('layer'+str(i))
  for j in range(IRIS_MLP.layers[i].numberOfPerceptrons):
    print('percepron'+str(j))
    for k in range(IRIS_MLP.layers[i].perceptrons[j].numberOfInputs):
      print(IRIS_MLP.layers[i].perceptrons[j].inputBranchs[k].w)

layer0
percepron0
1
percepron1
1
layer1
percepron0
0.6672
0.6672
percepron1
0.6672
0.6672
layer2
percepron0
0.6672
0.6672
percepron1
0.6672
0.6672


In [ ]:
(0.6 - 1.12*2.4*0.1)*1.12*2

0.7418879999999999

In [ ]:
(0.6-0.1*0.7418*2)*2

0.9032799999999999

In [ ]:
0.9032799999999999*0.3311*2

0.5981520159999999